<a href="https://colab.research.google.com/github/Seowon-Ji/Multi-target/blob/master/vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from collections import namedtuple
import torch
from torchvision import models


class Vgg16(torch.nn.Module):
    def __init__(self, requires_grad=False):
        super(Vgg16, self).__init__()
        vgg_pretrained_features = models.vgg16(pretrained=True).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        for x in range(4):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(4, 9):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(9, 16):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(16, 23):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        h = self.slice1(X)
        h_relu1_2 = h
        h = self.slice2(h)
        h_relu2_2 = h
        h = self.slice3(h)
        h_relu3_3 = h
        h = self.slice4(h)
        h_relu4_3 = h
        vgg_outputs = namedtuple("VggOutputs", ['relu1_2', 'relu2_2', 'relu3_3', 'relu4_3'])
        out = vgg_outputs(h_relu1_2, h_relu2_2, h_relu3_3, h_relu4_3)
        return out

class Vgg19(torch.nn.Module):
    def __init__(self, requires_grad=False):
        super(Vgg19, self).__init__()
        vgg_pretrained_features = models.vgg19(pretrained=True).features
        # self.vgg_pretrained_features = models.vgg19(pretrained=True).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        self.slice5 = torch.nn.Sequential()
        for x in range(2): #relu1_1
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(2, 7): #relu2_1
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(7, 15): #conv3_3
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(12, 21): #relu4_1
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        for x in range(21, 30): #relu5_1
            self.slice5.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X, indices=None):
        # if indices is None:
        #     indices = [2, 7, 12, 21, 30]
        # out = []
        # #indices = sorted(indices)
        # for i in range(indices[-1]):
        #     X = self.vgg_pretrained_features[i](X)
        #     if (i+1) in indices:
        #         out.append(X)
        #
        # return out

        h_relu1 = self.slice1(X)
        h_relu2 = self.slice2(h_relu1)
        h_relu3 = self.slice3(h_relu2)
        h_relu4 = self.slice4(h_relu3)
        h_relu5 = self.slice5(h_relu4)
        # out = [h_relu1, h_relu2, h_relu3, h_relu4, h_relu5]
        out =  h_relu3
        return out

def normalize_vgg(x):
    rgb_mean = torch.tensor([0.485, 0.456, 0.406]).cuda().view(1, 3, 1, 1)
    rgb_std = torch.tensor([0.229, 0.224, 0.225]).cuda().view(1, 3, 1, 1)
    y = (x - rgb_mean) / rgb_std
    return y


#
# if __name__ == '__main__':
#     vgg = Vgg19()
#     import ipdb; ipdb.set_trace()